### import dependencies

In [68]:
##### work in this repository was inspired by 
##### https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/blob/main/README_EN.md
##### https://www.youtube.com/watch?v=a99p_fAr6e4


### import dependencies

In [69]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd


# this is just to make sure that the results are reproducible to anyone that runs the code lol.
RANDOM_SEED = 42

#### data formatting
https://devqa.io/python-convert-csv-file-to-list/

In [70]:
import glob
labels = {0: "closed", 1: "open"}

Y_train = [] #1d array, (big_number, 1)
X_train = [] #2d array, (big_number, 42)
def data_append_train(data, X_train = X_train, Y_train = Y_train):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/train"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_train.append([float(num) for num in row])
                Y_train.append(label)
    
    print(X_train[-1])
    print("training set:")
    print(len(X_train))
    print(len(Y_train))    

Y_test = []
X_test = []
def data_append_test(data, X_test = X_test, Y_test = Y_test):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/test"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_test.append([float(num) for num in row])
                Y_test.append(label)
    
    print(X_test[-1])
    print("training set:")
    print(len(X_test))
    print(len(Y_test))    
#i'll only use the train test.
data = "data"
data_append_train(data)
data_append_test(data)




closed
['data/train/closed\\eye.csv']
1
[216.85504913330078, 152.74354934692383, 4.718387126922607, 300.8012008666992, 160.9013557434082, -11.97317361831665, 244.46121215820312, 160.7812786102295, -10.024803876876831, 275.7936096191406, 160.86087226867676, -15.670033693313599, 269.5697784423828, 157.55355834960938, -24.473342895507812, 237.7887725830078, 156.99408531188965, -17.94538140296936, 493.7444305419922, 180.18958568572998, 0.06788171827793121, 409.0971374511719, 173.22411060333252, -14.07525897026062, 464.72591400146484, 182.40240097045898, -13.882994651794434, 433.48480224609375, 176.8485403060913, -18.609195947647095, 441.343994140625, 172.87556648254395, -27.493326663970947, 473.0286407470703, 178.6200714111328, -22.001070976257324]
training set:
566
566
open
['data/train/open\\eye.csv']
1
[447.63343811035156, 28.05812358856201, -4.567684829235077, 484.0408706665039, 36.453402042388916, -7.2414761781692505, 459.1893768310547, 35.93154430389404, -9.6647310256958, 472.6679229

### since I already have the training dataset split, I just need to train!

In [71]:
# X_df_train = pd.DataFrame(X_train)
# Y_df_train = pd.DataFrame(Y_train)

# X_df_test = pd.DataFrame(X_test)
# Y_df_test = pd.DataFrame(Y_test)


X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
# 
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)


from sklearn.utils import shuffle

# the random_state is so that both shuffle perform the same.
X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_test,Y_test = shuffle(X_test, Y_test, random_state=0)
# X_train.shape, Y_train.shape
X_train, Y_train


(array([[ 2.10942459e+02,  1.76260071e+02,  2.64112639e+01, ...,
          3.26990395e+02,  1.78720236e+02,  5.03394365e+00],
        [ 6.31320477e+01,  1.98462238e+02,  4.23670578e+01, ...,
          1.31166334e+02,  1.39122362e+02, -2.56325459e+01],
        [ 2.94060850e+01,  2.17488499e+02,  2.33728623e+01, ...,
          1.44218750e+02,  2.09302411e+02,  5.02811491e+00],
        ...,
        [ 8.95617676e+01,  2.18508797e+02,  3.17117834e+01, ...,
          2.07774029e+02,  1.84710073e+02, -6.60983962e-03],
        [ 2.08185616e+02,  1.69075570e+02, -1.36326253e+00, ...,
          4.92963905e+02,  2.00928783e+02, -3.51337385e+01],
        [ 2.36047411e+01,  2.18941598e+02,  2.21065569e+01, ...,
          1.38865423e+02,  2.09952192e+02,  3.81412655e+00]]),
 array([0, 1, 1, ..., 1, 0, 1]))

In [72]:
NUM_CLASSES = 2
# model_save_path = "model/"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model/eye_model/model.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    #everytime the accuracy gets better, it saves
    save_freq='epoch',
)


#defining the model 

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras

In [74]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (1063, 36)
Shape of X_test: (817, 36)


In [75]:

model = Sequential([
    # I will just change the input 
    Dense(units=32, activation='relu', input_shape=(36,)),
    Dense(units=32, activation='relu',
        kernel_regularizer=keras.regularizers.l1_l2(0.05)),
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=16, activation='relu', 
        kernel_regularizer=keras.regularizers.l1_l2(0.01)),
    # Dropout(0.2),
    Dense(units=NUM_CLASSES, activation = "softmax")
])

In [76]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [77]:
#fitting
history = model.fit(X_train,Y_train, epochs=150, batch_size=32, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/150
34/34 [==============================] - 6s 160ms/step - loss: 12.4527 - accuracy: 0.6058 - val_loss: 9.7799 - val_accuracy: 0.4333
Epoch 2/150
34/34 [==============================] - 4s 134ms/step - loss: 9.1837 - accuracy: 0.6265 - val_loss: 8.4237 - val_accuracy: 0.4798
Epoch 3/150
34/34 [==============================] - 0s 10ms/step - loss: 7.7769 - accuracy: 0.6444 - val_loss: 7.7140 - val_accuracy: 0.4798
Epoch 4/150
34/34 [==============================] - 4s 134ms/step - loss: 6.6228 - accuracy: 0.6538 - val_loss: 6.6913 - val_accuracy: 0.4994
Epoch 5/150
34/34 [==============================] - 0s 9ms/step - loss: 5.6195 - accuracy: 0.6914 - val_loss: 5.5683 - val_accuracy: 0.4774
Epoch 6/150
34/34 [==============================] - 0s 10ms/step - loss: 4.8158 - accuracy: 0.6736 - val_loss: 4.7862 - val_accuracy: 0.4565
Epoch 7/150
34/34 [==============================] - 0s 13ms/step - loss: 4.1518 - accuracy: 0.7093 - val_loss: 4.2208 - val_accuracy: 0.4994
Epo